In [1]:
import sys
sys.path.append("../")

In [1]:
import pandas as pd
df = pd.read_csv(
    "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2019-20/players_raw.csv"
)
df2 = pd.read_csv(
    "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/f0cd9a537e4daff00a9a8b0d8019bc4e28da94d3/data/2019-20/players_raw.csv"
)
df = pd.merge(df[["first_name", "second_name", "total_points", "team_code", "element_type", "now_cost", "id"]],
              df2[["id", "total_points"]],
              on="id", suffixes=["", "_last_season"])
df.head()

old_expected_scores = df["total_points_last_season"] / 38
new_expected_scores = (0.9*old_expected_scores + df["total_points"]*0.1).fillna(0)  # weighted average points from last season and points last week
prices = df["now_cost"] / 10
positions = df["element_type"]
clubs = df["team_code"]
# so we can read the results
names = df["first_name"] + " " + df["second_name"]
# retrieve the team I picked last week

KeyError: 'total_points_last_season'

In [3]:
from fpl_opt.transfers import TransferOptimiser, MultiHorizonTransferOptimiser
from fpl_opt.selection import select_team

In [4]:
opt = TransferOptimiser(new_expected_scores, prices, prices, positions, clubs)

In [5]:
decisions, captain_decisions, sub_decisions = select_team(old_expected_scores.values, prices.values, positions.values, clubs.values)
player_indices = []
for i in range(len(decisions)):
    if decisions[i].value() == 1 or sub_decisions[i].value() == 1:
        print(names[i])
        player_indices.append(i)

Total expected score = 62.73684210526316
Edward Nketiah
David Luiz Moreira Marinho
Ryan Fraser
N'Golo Kanté
Luka Milivojevic
Lucas Digne
Gylfi Sigurdsson
Andrew Robertson
Virgil van Dijk
Mohamed Salah
Aymeric Laporte
Ederson Santana de Moraes
Shane Long
Lukasz Fabianski
Raúl Jiménez


In [6]:
transfer_in, transfer_out, starters, subs, captains = opt.solve(player_indices, budget_now=0, sub_factor=0.1)

In [7]:
for i in range(opt.num_players):
    if transfer_in[i].value() == 1:
        print("Transferred in: {} {} {}".format(names[i], prices[i], new_expected_scores[i]))
    if transfer_out[i].value() == 1:
        print("Transferred out: {} {} {}".format(names[i], prices[i], new_expected_scores[i]))

Transferred out: David Luiz Moreira Marinho 6.0 3.8842105263157896
Transferred in: Ricardo Domingos Barbosa Pereira 6.0 4.3578947368421055


In [8]:
for i in range(opt.num_players):
    if starters[i].value() == 1:
        print("{} {}".format(names[i], new_expected_scores[i]))

Ryan Fraser 4.386842105263158
Luka Milivojevic 4.131578947368421
Lucas Digne 4.342105263157896
Gylfi Sigurdsson 4.610526315789474
Ricardo Domingos Barbosa Pereira 4.3578947368421055
Andrew Robertson 5.244736842105263
Virgil van Dijk 5.726315789473684
Mohamed Salah 7.33421052631579
Aymeric Laporte 4.792105263157895
Ederson Santana de Moraes 4.702631578947369
Raúl Jiménez 4.486842105263158


In [9]:
for i in range(opt.num_players):
    if subs[i].value() == 1:
        print("{} {}".format(names[i], new_expected_scores[i]))

Edward Nketiah 0.23684210526315788
N'Golo Kanté 2.7473684210526317
Shane Long 1.5631578947368423
Lukasz Fabianski 3.486842105263158


In [16]:
import numpy as np
opt = MultiHorizonTransferOptimiser(np.array([new_expected_scores, new_expected_scores, new_expected_scores, new_expected_scores]),
                                    prices, prices, positions, clubs, 4)

In [17]:
transfer_in, transfer_out, starters, subs, captains = opt.solve(player_indices, budget_now=0, sub_factor=0.1)

<class 'pulp.pulp.LpAffineExpression'>
<class 'pulp.pulp.LpAffineExpression'>
<class 'pulp.pulp.LpAffineExpression'>
<class 'pulp.pulp.LpAffineExpression'>


In [18]:
for tin, tout in zip(transfer_in, transfer_out):
    for i in range(opt.num_players):
        if tin[i].value() == 1:
            print("Transferred in: {} {} {}".format(names[i], prices[i], new_expected_scores[i]))
        if tout[i].value() == 1:
            print("Transferred out: {} {} {}".format(names[i], prices[i], new_expected_scores[i]))

Transferred out: David Luiz Moreira Marinho 6.0 3.8842105263157896
Transferred in: Ricardo Domingos Barbosa Pereira 6.0 4.3578947368421055
Transferred out: Mohamed Salah 12.5 7.33421052631579
Transferred in: Raheem Sterling 12.0 7.542105263157895
Transferred out: Edward Nketiah 4.5 0.23684210526315788
Transferred in: Connor Wickham 4.5 0.3131578947368421
Transferred in: Jordan Pickford 5.5 4.513157894736842
Transferred out: Jordan Pickford 5.5 4.513157894736842
